In [1]:
''' Classifier
1. Input Image
2. Model (CNN, Transformer, ResNet) - DNN, ML Model (Random forest, XGBoost)
3. 1-20 output number of qubit, type identification (6)
4. Calculate accuracy
5. Feature engineering for LLM
'''

# Code goes here

# start generate me a code to 

import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from tqdm import tqdm


In [2]:
# dataset path
DATASET_PATH = "generated_images_qutip_grayscale"

BATCH_SIZE = 16
NUM_EPOCHS = 50
LEARNING_RATE = 0.00005
IMG_SIZE = 224

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Standard normalization
])

dataset = ImageFolder(root=DATASET_PATH, transform=transform)

#split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# dataloader 
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)

NUM_CLASSES = len(dataset.classes)
print(f"Detected {NUM_CLASSES} classes.")




Detected 19 classes.


In [ ]:
import torchvision.models as models

# model = models.resnet50(pretrained=True)

# model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

# generate me a model class to clasify images using resnet50
class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.model = models.vgg16(pretrained=True)
        self.sequential = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        x = self.sequential(x)
        return x

model = ResNet50(NUM_CLASSES)
model = model.to(device)

TypeError: 'module' object is not callable

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [6]:
def evaluate_model(model, val_loader):
    model.eval()
    correct, total = 0,0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    return 100.0 * (correct/ total)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels  = images.to(device), labels.to(device)
            
            # forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # backward pass
            loss.backward()
            optimizer.step()
            
            # track loss and accuracy
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
        train_acc = 100.0 * correct / total
        val_acc = evaluate_model(model, val_loader=val_loader)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
            
            
train_model(model, train_loader, val_loader, criterion, optimizer=optimizer, num_epochs=NUM_EPOCHS)

Epoch 1/50: 100%|██████████| 44/44 [00:04<00:00, 10.13it/s]


Epoch [1/50] | Loss: 2.8790 | Train Acc: 8.38% | Val Acc: 9.83%


Epoch 2/50: 100%|██████████| 44/44 [00:03<00:00, 11.40it/s]


Epoch [2/50] | Loss: 2.5739 | Train Acc: 18.21% | Val Acc: 16.18%


Epoch 3/50: 100%|██████████| 44/44 [00:03<00:00, 11.18it/s]


...

Epoch [48/50] | Loss: 0.5440 | Train Acc: 81.79% | Val Acc: 52.60%


Epoch 49/50: 100%|██████████| 44/44 [00:03<00:00, 11.41it/s]


Epoch [49/50] | Loss: 0.5479 | Train Acc: 81.21% | Val Acc: 51.45%


Epoch 50/50: 100%|██████████| 44/44 [00:03<00:00, 11.48it/s]


Epoch [50/50] | Loss: 0.5367 | Train Acc: 82.66% | Val Acc: 51.45%


In [ ]:
''' Chain of thought
1. Generate dataset (input - output)
2. Generate CoT (CoT + output)
3. Linear space -10 to 10

CoT:
- First I should identify from the image that linear space is range from -10 to 10 in X axis and -10 to 10 in Y axis, therefore linspace is (-10, 10)
- (Type) (Equation wigner -> type)
- (Type of parameters) 
- (Number of qubit) (Equation wigner -> number of qubit)
- (Parameters)
Output:

'''
# Code goes here
# rho_coherent = coherent_dm(10, 7)

NameError: name 'coherent_dm' is not defined

In [1]:
''' Florence 2 Finetuning 
For trial: https://huggingface.co/spaces/gokaygokay/Florence-2

'''

' Florence 2 Finetuning \nFor trial:'